# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 9:41AM,241053,12,8234594,Doctors Scientific Organica,Released
1,Jul 5 2022 9:36AM,241047,12,HH6331,Hush Hush,Released
2,Jul 5 2022 9:36AM,241047,12,HH6338,Hush Hush,Released
3,Jul 5 2022 9:36AM,241047,12,HH6324,Hush Hush,Released
4,Jul 5 2022 9:36AM,241047,12,HH6338.1,Hush Hush,Released
5,Jul 5 2022 9:36AM,241047,12,HH6339,Hush Hush,Released
6,Jul 5 2022 9:36AM,241047,12,HH-35385,Hush Hush,Released
7,Jul 5 2022 9:36AM,241047,12,HH-35394,Hush Hush,Released
8,Jul 5 2022 9:36AM,241047,12,HH-35417,Hush Hush,Released
9,Jul 5 2022 9:36AM,241047,12,HH-35442,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,241044,Released,54
37,241045,Released,2
38,241047,Released,16
39,241048,Released,65
40,241053,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241044,NaN,54.0
241045,NaN,2.0
241047,NaN,16.0
241048,NaN,65.0
241053,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241002,27.0,28.0
241004,0.0,59.0
241005,0.0,54.0
241006,0.0,55.0
241008,0.0,60.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241002,27,28
241004,0,59
241005,0,54
241006,0,55
241008,0,60


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241002,27,28
1,241004,0,59
2,241005,0,54
3,241006,0,55
4,241008,0,60


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241002,27,28
1,241004,,59
2,241005,,54
3,241006,,55
4,241008,,60


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica
1,Jul 5 2022 9:36AM,241047,12,Hush Hush
17,Jul 5 2022 9:33AM,241048,12,Hush Hush
82,Jul 5 2022 9:18AM,241045,12,Hush Hush
84,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation
138,Jul 5 2022 9:09AM,241041,10,ISDIN Corporation
198,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.
200,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc."
201,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation
261,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica,,1
1,Jul 5 2022 9:36AM,241047,12,Hush Hush,,16
2,Jul 5 2022 9:33AM,241048,12,Hush Hush,,65
3,Jul 5 2022 9:18AM,241045,12,Hush Hush,,2
4,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation,,54
5,Jul 5 2022 9:09AM,241041,10,ISDIN Corporation,,60
6,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,,2
7,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",1,
8,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,,60
9,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,,62


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica,1,
1,Jul 5 2022 9:36AM,241047,12,Hush Hush,16,
2,Jul 5 2022 9:33AM,241048,12,Hush Hush,65,
3,Jul 5 2022 9:18AM,241045,12,Hush Hush,2,
4,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation,54,
5,Jul 5 2022 9:09AM,241041,10,ISDIN Corporation,60,
6,Jul 5 2022 9:05AM,241043,10,Eywa Pharma Inc.,2,
7,Jul 5 2022 9:03AM,241042,19,"GCH Granules USA, Inc.",,1
8,Jul 5 2022 8:57AM,241040,10,ISDIN Corporation,60,
9,Jul 5 2022 8:57AM,241039,10,ISDIN Corporation,62,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica,1,
1,Jul 5 2022 9:36AM,241047,12,Hush Hush,16,
2,Jul 5 2022 9:33AM,241048,12,Hush Hush,65,
3,Jul 5 2022 9:18AM,241045,12,Hush Hush,2,
4,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation,54,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica,1.0,NaN
1,Jul 5 2022 9:36AM,241047,12,Hush Hush,16.0,NaN
2,Jul 5 2022 9:33AM,241048,12,Hush Hush,65.0,NaN
3,Jul 5 2022 9:18AM,241045,12,Hush Hush,2.0,NaN
4,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation,54.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 9:41AM,241053,12,Doctors Scientific Organica,1.0,0.0
1,Jul 5 2022 9:36AM,241047,12,Hush Hush,16.0,0.0
2,Jul 5 2022 9:33AM,241048,12,Hush Hush,65.0,0.0
3,Jul 5 2022 9:18AM,241045,12,Hush Hush,2.0,0.0
4,Jul 5 2022 9:09AM,241044,10,ISDIN Corporation,54.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7953800,1329.0,59.0
12,964193,84.0,0.0
19,241042,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7953800,1329.0,59.0
1,12,964193,84.0,0.0
2,19,241042,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1329.0,59.0
1,12,84.0,0.0
2,19,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1329.0
1,12,Released,84.0
2,19,Released,0.0
3,10,Executing,59.0
4,12,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Executing,59.0,0.0,1.0
Released,1329.0,84.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Executing,59.0,0.0,1.0
1,Released,1329.0,84.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Executing,59.0,0.0,1.0
1,Released,1329.0,84.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()